In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

BATCH_SIZE = 4
EPOCHS = 2

train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 0.2,shear_range = 0.2,
    zoom_range = 0.2,width_shift_range = 0.2,
    height_shift_range = 0.2, validation_split = 0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\Divyansh\Downloads\Drowsiness\mrlEyes_2018_01\Prepared Data\train',
                                target_size = (80,80), batch_size = BATCH_SIZE, 
                                class_mode = 'categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\Divyansh\Downloads\Drowsiness\mrlEyes_2018_01\Prepared Data\train',
                                target_size = (80,80), batch_size = BATCH_SIZE, 
                                class_mode = 'categorical',subset='validation' )


test_datagen = ImageDataGenerator(rescale = 1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Divyansh\Downloads\Drowsiness\mrlEyes_2018_01\Prepared Data\test',
                                target_size=(80,80), batch_size = BATCH_SIZE, class_mode='categorical')



bmodel = InceptionV3(include_top = False, weights = 'imagenet', 
                     input_tensor = Input(shape = (80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation = 'relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation = 'softmax')(hmodel)

model = Model(inputs = bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False


from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau


checkpoint = ModelCheckpoint(r'C:\Users\Divyansh\Downloads\Drowsiness\mrlEyes_2018_01\model',
                            monitor = 'val_loss', save_best_only = True, verbose = 3)

earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, 
                          verbose= 3, restore_best_weights = True)


learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks = [checkpoint, earlystop, learning_rate]



model.compile(optimizer = 'Adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])


model.fit_generator(train_data,steps_per_epoch = train_data.samples// BATCH_SIZE,
                   validation_data = validation_data,
                   validation_steps = validation_data.samples// BATCH_SIZE,
                   callbacks = callbacks,
                    epochs = EPOCHS)


# Model Evaluation


acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)


acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)


acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

Found 64706 images belonging to 2 classes.
Found 16175 images belonging to 2 classes.
Found 4017 images belonging to 2 classes.


C:\Users\Divyansh\AppData\Local\Temp\ipykernel_9464\2894322250.py:65: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch = train_data.samples// BATCH_SIZE,


Epoch 1/2
16176/16176 [==============================] - ETA: 0s - loss: 0.1957 - accuracy: 0.9256
Epoch 1: val_loss improved from inf to 0.31728, saving model to C:\Users\Divyansh\Downloads\Drowsiness\mrlEyes_2018_01\model
INFO:tensorflow:Assets written to: C:\Users\Divyansh\Downloads\Drowsiness\mrlEyes_2018_01\model\assets


INFO:tensorflow:Assets written to: C:\Users\Divyansh\Downloads\Drowsiness\mrlEyes_2018_01\model\assets


16176/16176 [==============================] - 907s 56ms/step - loss: 0.1957 - accuracy: 0.9256 - val_loss: 0.3173 - val_accuracy: 0.8831 - lr: 0.0010
Epoch 2/2
  724/16176 [>.............................] - ETA: 9:26 - loss: 0.1776 - accuracy: 0.9313